In [ ]:
import yfinance as yf

meli34 = yf.Ticker("MELI34.SA")
df_meli34 = meli34.history(start="2021-01-01",end="2023-12-30")

In [ ]:
total_rows = len(df_meli34)
legs = 21

df_train = df_meli34.iloc[:total_rows - legs]
df_test = df_meli34.iloc[total_rows - legs:]

In [ ]:
training_set = df_train['Close'].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set.reshape(-1,1))


In [ ]:
intervalo_entre_os_dias = 15 # Dias vistos para trás

In [ ]:



X_train = []
y_train = []

for i in range(intervalo_entre_os_dias, len(training_set)):
  X_train.append(training_set_scaled[i-intervalo_entre_os_dias:i, 0]) #vai pegar sempre os 60 dias para tras
  y_train.append(training_set_scaled[i, 0]) #vai pegar sempre todos os valores do intervalo

import numpy as np

X_train, y_train = np.array(X_train), np.array(y_train)

     

X_train.shape
     


In [ ]:
batch_size = len(X_train) # Quantidade de registros que passará para a rede
time_steps = intervalo_entre_os_dias # Quantidade de dias que serão usados para prever o próximo
features = 1 

X_train = np.reshape(X_train, (batch_size, time_steps, features)) # Transforma o array para ter 3 dimensões 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

rnn = Sequential()


In [ ]:
'''
return_sequences=True -> É usado para utilizar o backpropregation em todas as camadas
'''
rnn.add(LSTM(units=50, return_sequences=True, input_shape=(time_steps, features)))
'''
Dropout -> É usado para evitar o overfitting quando ocorre a backpropragação removendo no caso 50% dos neurônios
'''
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50))
rnn.add(Dropout(0.5))

'''
Dense conecta todas as camadas para retornar 1 valor
'''
rnn.add(Dense(units=1))

In [ ]:
rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
'''
Epochs -> Quantidade de vezes que o modelo vai treinar
'''
rnn.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
import pandas as pd
total_rows = len(df_meli34)
legs = 21

df_train = df_meli34.iloc[:total_rows - legs]
df_test = df_meli34.iloc[total_rows - legs:]

fechamento_real = df_test['Close'].values

dataset_total = pd.concat((df_meli34['Close'], df_test['Close']), axis=0)
inputs = dataset_total[len(dataset_total) - len(df_test) - intervalo_entre_os_dias:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)



In [ ]:
X_test = []
for i in range(intervalo_entre_os_dias, len(inputs)):
  X_test.append(inputs[i-intervalo_entre_os_dias:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
predict = rnn.predict(X_test)

variacao_final = sc.inverse_transform(predict)
test = df_test['Close'].values

from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(test, variacao_final)

In [ ]:
import matplotlib.pyplot as plt
name = "meli34"
fechamento_real = df_test['Close'].values

plt.plot(fechamento_real, color = 'red', label = f'Dados Reais de Ações da {name}')
plt.plot(variacao_final, color = 'blue', label = f'Dados Previstos de Ações do {name}')
plt.title('Previsão de Preços de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preços de Ações')
plt.legend()
plt.show()